# QCoDeS Example with QDevil_QDac_channels

In [ ]:
import qcodes as qc
import numpy as np
from time import sleep

from qcodes.instrument_drivers.QDevil.QDevil_QDac_channels import QDac

In [ ]:
# Connect to the instrument
# The initialisation takes some time because of the 0.2-0.5 sec settling time of 
# the current sensors - and they are all being read at startup. 
# An "update_currents=False" parameter can be added to the initialising call
# in order to skip reading the current sensors
qdac = QDac(name='qdac', address='ASRL2::INSTR')

## Basic QDAC Usage

The QDevil_Qdac_channels driver supports controlling each individual BNC output channel. Each output channel has six parameters:
  * v: DC voltage
  * vrange: DC voltage range
  * i: Current out (read-only)
  * irange: Current measurement range
  * slope: Maximum ramp rate for an output channel
  * sync: Sync output assigned to a channel 
  * sync_delay: Sync delay delay 
  * sync_duration: Sync duration
  
The slope is the (maximal) slope in V/s that the channel can allow its voltage to change by. By default, all channels have a slope of "Inf". The slope can be changed dynamically, but no more than 8 channels can have a finite slope at any given time (this is due to hardware limitations).

In addition this driver supports reading the internal temperature sensorsand pretty printing the state of all channels. 

In [ ]:
# The DC voltage may directly be set and gotten
qdac.ch01.v.set(-1)
print('Channel 1 voltage: {} {}'.format(qdac.ch01.v.get(), qdac.ch01.v.unit))

In [ ]:
# Current out is the current flowing through the channel this is read-only. 
print(qdac.ch01.i.get(), qdac.ch01.i.unit)

In [ ]:
# The current range can be either 0 to 1 μA or 0 to 100 μA. But it depends on the voltage range.
# The combination of high current range and low voltage range is not allowed
# Unless the output current is zero, changing the the current range may give a small spike on the output.
print(qdac.ch01.irange.get())
# This is set with either 0 (0 to 1 μA) or 1 (0 to 100 μA) 
qdac.ch01.irange.set(1)

In [ ]:
# Multiple channels can be addressed simultaneously via the 'channels' list, by use of slicing.
# Note that numbering goes from 0 to N, where N is the number of channels. 
# This will query voltages of all channels of a 24 channel QDAC 
# Note that index 0 refer to channel 01, and so on 
print(qdac.channels[0:24].v.get())

In [ ]:
# Similarly, we may set them
qdac.channels[0:24].v.set(-0.9)

In [ ]:
# The maximal voltage change (in V/s) may be set for each channel
qdac.ch01.slope.set(1)
qdac.ch02.slope.set(2)
# An overview may be printed (all other channels have 'Inf' slope)
qdac.printslopes()

In [ ]:
# now setting channel 1 and 2 voltages will cause slow ramping to 0V
qdac.ch01.v.set(0)
qdac.ch02.v.set(0)

In [ ]:
# Note that only 8 (or fewer) channels can have finite slopes at one time
# To make space for other channels, set the slope to inifite
qdac.ch01.slope('Inf')
qdac.printslopes()

In [ ]:
# To each channel one may assign a sync channel:
qdac.ch01.sync(1)  # sync output 1 will fire a 10 ms 5 V pulse when ch02 ramps
# note that even if no visible ramp is performed (i.e. ramping from 1 V to 1 V), a pulse is still fired.

# The sync pulse settings can be modified
qdac.ch01.sync_delay(0)  # The sync pulse delay (s)
qdac.ch01.sync_duration(25e-3)  # The sync pulse duration (s). Default is 10 ms.

In [ ]:
# syncs are unassigned by assigning sync 0
qdac.ch02.sync(0)

# Changing the voltage range - attention!

The v_range parameter is controlling an attenuator. Upon changing the vrange, the attenuator is **immediately** applied (or revoked). The driver will re-adjust the output voltage in order to keep it constannt, but a spike will always occur. If the set voltage is outside the range of the low range and the switch is from high to low range, the output will be clipped. So it is recommended always to set the output to zero before chaning the voltage range.
Further, the current range may be automatically flipped, if vrange is switched from high to low range (i.e. 0 to 1) and the current range is in the high range (1, default).

In [ ]:
# Here is a small example showing what to look out for
#
qdac.ch01.vrange.set(0)  # Attenuation OFF (the default), high volage range
qdac.ch01.v.set(1.5)     # Set the voltage to outside the low voltage range
qdac.ch01.vrange.set(1)  # Attenuation ON, low voltage range
print(qdac.ch01.v.get()) # Returns approximately 1.1V as the output is clipped to the low range limit
qdac.ch01.vrange.set(0)  # Attenuation off, high voltage range
print(qdac.ch01.v.get()) # Returns approximately 1.1V, unchanged - but there will be a spike

## Overview of channel settings

The driver provides a method for pretty-printing the state of all channels. On startup, all channels are queried for voltage and current across them, but the current query is very slow (blame the hardware).

The pretty-print method may or may not **update** the values for the currents, depending on the value of the `update_currents` flag. Each current reading takes some 200-500 ms, so updating all current values takes about 14-24 seconds depending on the number of channels.

In [ ]:
qdac.print_overview(update_currents=True)

### Temperature sensors

Physically, the QDac consists of either three or six boards each hosting eight channels. On three locations on each board, a temperature sensors is placed. These provide read-only parameters, named `tempX_Y` where `X` is the board number (0-2, or 0-5) and `Y` the sensor number (0-2). 

In [ ]:
print(qdac.temp0_0.get(), qdac.temp0_0.unit)
print(qdac.temp2_1.get(), qdac.temp0_0.unit)

In [ ]:
qdac.close()